# Economo-2018 - A DataJoint example
This notebook presents data and results associated with the following papers:

>Michael N. Economo, Sarada Viswanathan, Bosiljka Tasic, Erhan Bas, Johan Winnubst, Vilas Menon, Lucas T. Graybuck,Thuc Nghi Nguyen, Kimberly A. Smith, Zizhen Yao, Lihua Wang, Charles R. Gerfen, Jayaram Chandrashekar, Hongkui Zeng, Loren L. Looger & Karel Svoboda. "Distinct descending motor cortex
pathways and their roles in movement" (2018) Nature (https://doi.org/10.1038/s41586-018-0642-9)

Original data is publically available at: doi: 10.25378/janelia.7007846

The data in original MATLAB format (.mat) have been ingested into a DataJoint data pipeline presented below. This notebook demonstrates the queries, processing, and reproduction of several figures from the paper.

In [1]:
from datetime import datetime
import os
os.chdir('..')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import datajoint as dj
from pipeline import (reference, subject, acquisition, analysis,
                      extracellular, behavior, utilities)

Connecting root@127.0.0.1:3306


## Data queries, conditioning, and reconstruction of Extended Data Figure 5

#### Queries of unit spike times and inter-spike interval, categorized by:
+ Cell type: PTupper vs. PTlower
+ Trial type: left-lick vs. right-lick
+ Hemisphere: left (contralateral to injection site) vs. right (ipsilateral to injection site)

In [2]:
# define trial restrictors
leftlick_trial = {'trial_type': 'lick left', 'trial_is_good': True, 'trial_response': 'correct'}
rightlick_trial = {'trial_type': 'lick right', 'trial_is_good': True, 'trial_response': 'correct'}

In [3]:
# units
good_units_filt = ['unit_quality="Excellent"', 'unit_quality="Good"', 'unit_quality="Single"']

lh_PTupper_units = (extracellular.UnitSpikeTimes & 'unit_cell_type="PTupper"' & 'hemisphere="left"' & good_units_filt)
lh_PTlower_units = (extracellular.UnitSpikeTimes & 'unit_cell_type="PTlower"' & 'hemisphere="left"' & good_units_filt)

rh_PTupper_units = (extracellular.UnitSpikeTimes & 'unit_cell_type="PTupper"' & 'hemisphere="right"' & good_units_filt)
rh_PTlower_units = (extracellular.UnitSpikeTimes & 'unit_cell_type="PTlower"' & 'hemisphere="right"' & good_units_filt)

In [4]:
print(f'Left-ALM PTupper unit counts: {len(lh_PTupper_units)}')
print(f'Left-ALM PTlower unit counts: {len(lh_PTlower_units)}')
print(f'Right-ALM PTupper unit counts: {len(rh_PTupper_units)}')
print(f'Right-ALM PTlower unit counts: {len(rh_PTlower_units)}')

Left-ALM PTupper unit counts: 65
Left-ALM PTlower unit counts: 46
Right-ALM PTupper unit counts: 0
Right-ALM PTlower unit counts: 32


In [5]:
# Specify segmentation setting to align to "cue-onset" event
seg_param_key = (analysis.TrialSegmentationSetting & {'event': 'cue_start'}).fetch1()

#### Per unit, perform:
+ extract spike-times and psth for all trials for left-lick and right-lick
+ compute inter-spike intervals for all spikes in all trials
+ exclude results with less than 50 correct trial-repsonse

In [6]:
# per unit, extract spike-times and psth
correct_trial_count_thresh = 50  # 50 correct trial of each left/right type
def get_psth_isi(unit_key):
    l_spks, l_psths = (extracellular.TrialSegmentedUnitSpikeTimes * extracellular.PSTH & unit_key & seg_param_key 
                       & (acquisition.TrialSet.Trial & leftlick_trial)).fetch('segmented_spike_times', 'psth')
    r_spks, r_psths = (extracellular.TrialSegmentedUnitSpikeTimes * extracellular.PSTH & unit_key & seg_param_key
                       & (acquisition.TrialSet.Trial & rightlick_trial)).fetch('segmented_spike_times', 'psth')
    
    if len(l_spks) < correct_trial_count_thresh or len(r_spks) < correct_trial_count_thresh:
        return None
        
    out = dict(isi=np.hstack(np.diff(spk) for spk in np.hstack([l_spks, r_spks])), 
               l_psth=np.vstack(psth for psth in l_psths).mean(axis=0),
               r_psth=np.vstack(psth for psth in r_psths).mean(axis=0))
    return out

In [7]:
lh_PTupper_units_psth = [v for v in [get_psth_isi(u) for u in lh_PTupper_units] if v]
lh_PTlower_units_psth = [v for v in [get_psth_isi(u) for u in lh_PTlower_units] if v]
rh_PTupper_units_psth = [v for v in [get_psth_isi(u) for u in rh_PTupper_units] if v]
rh_PTlower_units_psth = [v for v in [get_psth_isi(u) for u in rh_PTlower_units] if v]

In [8]:
print(f'Left-ALM PTupper unit counts: {len(lh_PTupper_units_psth)}')
print(f'Left-ALM PTlower unit counts: {len(lh_PTlower_units_psth)}')
print(f'Right-ALM PTupper unit counts: {len(rh_PTupper_units_psth)}')
print(f'Right-ALM PTlower unit counts: {len(rh_PTlower_units_psth)}')

Left-ALM PTupper unit counts: 62
Left-ALM PTlower unit counts: 46
Right-ALM PTupper unit counts: 0
Right-ALM PTlower unit counts: 32


#### Plot PSTH and ISI

In [9]:
def split_list(arr, chunk_size):
    for s in range(0, len(arr), chunk_size):
        yield arr[s:s+chunk_size]

In [10]:
edges = np.arange(-.025, .025, .0005)
def plot_psth_isi(isi_psth_dict, axs):
    psth_time = np.linspace(-1 * float(seg_param_key['pre_stim_duration']), 
                            float(seg_param_key['post_stim_duration']),
                            len(isi_psth_dict['l_psth']))
    
    axs[0].hist(np.vstack([isi_psth_dict['isi'], -isi_psth_dict['isi']]), 
                                                      bins=len(edges), range=(edges[0], edges[-1]))
    
    axs[1].plot(psth_time, isi_psth_dict['l_psth'], 'r')
    axs[1].plot(psth_time, isi_psth_dict['r_psth'], 'b')
    

In [ ]:
fig, axes = plt.subplots(2, 4)
for axs, isi_psth_dict in zip(axes.T, lh_PTupper_units_psth[:5]):
    plot_psth_isi(isi_psth_dict, axs)

In [ ]:
extracellular.UnitSpikeTimes & good_units_filt

In [ ]:
set(extracellular.UnitSpikeTimes.fetch('unit_quality'))

In [ ]:
a = (extracellular.TrialSegmentedUnitSpikeTimes * extracellular.PSTH & seg_param_key & leftlick_trial).fetch(
    'segmented_spike_times', 'psth')

In [ ]:
b = dict(zip(('spks', 'psth'), a))

In [ ]:
b['psth'].shape

In [ ]:
acquisition.TrialSet.Trial & 'trial_is_good=1'

In [ ]:
set((acquisition.TrialSet.Trial & 'trial_is_good=1').fetch('trial_response'))